In [19]:
import pandas as pd

data = pd.read_csv("train.csv")
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [20]:
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
import torch

model = model.to("cuda")

In [22]:
data = data[["comment_text","toxic"]]
data = data[0:1000]

In [23]:
from sklearn.model_selection import train_test_split

X = list(data["comment_text"])
y = list(data["toxic"])
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, truncation=True, padding=True, max_length=512)
X_val_tokenized = tokenizer(X_val, truncation=True, padding=True, max_length=512)

In [24]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels=None):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, index):
    item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
    if self.labels:
      item["labels"] = torch.tensor(self.labels[index])
    return item

  def __len__(self):
    return len(self.encodings["input_ids"])

In [25]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [26]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

def compute_metrics(p):
  pred, labels = p
  pred = np.argmax(pred, axis=1)

  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred)
  precision = precision_score(y_true=labels, y_pred=pred)
  f1 = f1_score(y_true=labels,y_pred=pred)

  return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

In [27]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=3,
    per_device_train_batch_size=8
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [28]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=300, training_loss=0.18528519948323569, metrics={'train_runtime': 145.8241, 'train_samples_per_second': 16.458, 'train_steps_per_second': 2.057, 'total_flos': 631466532864000.0, 'train_loss': 0.18528519948323569, 'epoch': 3.0})

In [29]:
trainer.evaluate()

{'eval_loss': 0.24183082580566406,
 'eval_accuracy': 0.945,
 'eval_recall': 0.7619047619047619,
 'eval_precision': 0.7272727272727273,
 'eval_f1': 0.7441860465116279,
 'eval_runtime': 3.6469,
 'eval_samples_per_second': 54.841,
 'eval_steps_per_second': 6.855,
 'epoch': 3.0}

In [30]:
text = "That point was good"

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize input and move to the same device
inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
inputs = {key: val.to(device) for key, val in inputs.items()}

# Forward pass
outputs = model(**inputs)

# Get predictions
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()

print(predictions)

[[0.9977316  0.00226842]]


In [31]:
import numpy as np

np.set_printoptions(suppress=True)

In [32]:
trainer.save_model("CustomModel")